In [8]:
import os
import numpy as np
from tqdm import tqdm

from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

In [13]:
def bbox_drawer(img_dir, label_dir, out_dir, names):
    img_files = os.listdir(img_dir)
    no_label_imgs = []
    no_label_img_cnt = 0
    for img_file in tqdm(img_files):
        label_file = f'{img_file.rsplit(".", 1)[0]}.txt'
        
        ext = img_file.rsplit(".", 1)[-1]
        if ext.lower() not in ('jpg', 'jpeg', 'png'):
            continue
        
        img_file_path = img_dir / img_file
        label_file_path = label_dir / label_file
        pil_img = Image.open(img_file_path)
        pil_draw = ImageDraw.Draw(pil_img)
        width, height = pil_img.size

        try:
            labels = np.loadtxt(label_file_path)
        except:
            no_label_img_cnt += 1
            no_label_imgs.append(img_file)
            labels = np.array([])

        if labels.size > 0:
            if labels.ndim == 1:
                labels = np.expand_dims(labels, 0)
            x1 = (labels[:, 1] - labels[:, 3] / 2) * width
            y1 = (labels[:, 2] - labels[:, 4] / 2) * height
            x2 = (labels[:, 1] + labels[:, 3] / 2) * width
            y2 = (labels[:, 2] + labels[:, 4] / 2) * height

            for label in np.stack((labels[:, 0], x1, y1, x2, y2), axis=1).tolist():
                cat, xx1, yy1, xx2, yy2 = label
                pil_draw.rectangle((xx1, yy1, xx2, yy2), outline='red', width=3)
                pil_draw.text((xx1, yy1 - 22), f'{names[int(label[0])]}', fill='red', font=ImageFont.load_default())
        
        img_out_file_path = out_dir / img_file
        pil_img.save(img_out_file_path)

    print(f'{no_label_img_cnt} image file dont have labels')
    return no_label_imgs, no_label_img_cnt

In [14]:
# names = {0: 'Buoy', 1: 'Boat', 2: 'Channel Marker', 3: 'Speed Warning Sign'}
# names = {0: 'Vessel', 1: 'Boat', 2: 'Buoy', 3: 'etc'}
names = {0: 'Vessel'}

In [16]:
for root, dirs, files in os.walk("/data01/HiNAS-DATA/PROCESSED-IR-DATA/"):
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        
        IMG_DIR = Path(os.path.join(dir_path, "images"))
        LABEL_DIR = Path(os.path.join(dir_path, "labels"))
        OUT_DIR = Path(os.path.join(dir_path, "masked_images"))
        
        OUT_DIR.mkdir(exist_ok=True)
        
        print(IMG_DIR, LABEL_DIR, OUT_DIR)
        
        bbox_drawer(IMG_DIR, LABEL_DIR, OUT_DIR, names)

    dirs[:] = []

/data01/HiNAS-DATA/PROCESSED-IR-DATA/YMwinner/images /data01/HiNAS-DATA/PROCESSED-IR-DATA/YMwinner/labels /data01/HiNAS-DATA/PROCESSED-IR-DATA/YMwinner/masked_images


100%|██████████| 713/713 [00:20<00:00, 35.34it/s]


0 image file dont have labels
/data01/HiNAS-DATA/PROCESSED-IR-DATA/ccnuri/images /data01/HiNAS-DATA/PROCESSED-IR-DATA/ccnuri/labels /data01/HiNAS-DATA/PROCESSED-IR-DATA/ccnuri/masked_images


100%|██████████| 228/228 [00:07<00:00, 29.47it/s]


0 image file dont have labels
/data01/HiNAS-DATA/PROCESSED-IR-DATA/2939/images /data01/HiNAS-DATA/PROCESSED-IR-DATA/2939/labels /data01/HiNAS-DATA/PROCESSED-IR-DATA/2939/masked_images


...

0 image file dont have labels
/data01/HiNAS-DATA/PROCESSED-IR-DATA/HYUNDAIGLOVIS/images /data01/HiNAS-DATA/PROCESSED-IR-DATA/HYUNDAIGLOVIS/labels /data01/HiNAS-DATA/PROCESSED-IR-DATA/HYUNDAIGLOVIS/masked_images


100%|██████████| 240/240 [00:03<00:00, 67.49it/s]


1 image file dont have labels
/data01/HiNAS-DATA/PROCESSED-IR-DATA/Seaspan/images /data01/HiNAS-DATA/PROCESSED-IR-DATA/Seaspan/labels /data01/HiNAS-DATA/PROCESSED-IR-DATA/Seaspan/masked_images


100%|██████████| 39/39 [00:00<00:00, 57.55it/s]

0 image file dont have labels
